<a href="https://colab.research.google.com/github/sowmR/ML-LinearRegression/blob/main/LinearRegression_scratch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.metrics import mean_squared_error, r2_score

In [ ]:
dataset = pd.read_csv('./sample_data/insurance_costs.csv')
print(dataset.shape)
print(dataset.columns)

(1338, 6)
Index(['Unnamed: 0', 'age', 'sex', 'children', 'region', 'charges'], dtype='object')


In [ ]:
dataset.drop('Unnamed: 0', axis=1,inplace=True)
features = list(dataset.columns)
target = 'charges'
print(features)
print(dataset.head(5))


['age', 'sex', 'children', 'region', 'charges']
   age     sex  children     region      charges
0   19  female         0  southwest  16884.92400
1   18    male         1  southeast   1725.55230
2   28    male         3  southeast   4449.46200
3   33    male         0  northwest  21984.47061
4   32    male         0  northwest   3866.85520


# Data Preprocessing

In [ ]:
dataset = pd.get_dummies(dataset)
dataset.head(5)

,age,children,charges,sex_female,sex_male,region_northeast,region_northwest,region_southeast,region_southwest
0,19,0,16884.92400,1,0,0,0,0,1
1,18,1,1725.55230,0,1,0,0,1,0
2,28,3,4449.46200,0,1,0,0,1,0
3,33,0,21984.47061,0,1,0,1,0,0
4,32,0,3866.85520,0,1,0,1,0,0


# Data Analysis

Analyse the data to understand the relation between features and target. 

In [ ]:
# import seaborn as sns
# sns.pairplot(dataset,hue='charges')

In [ ]:
targetData = list(dataset[target])
featureDf =  dataset.copy()
featureDf.drop(target,axis=1, inplace=True)
featureDf.shape

(1338, 8)

In [ ]:
featureData = featureDf.values
x_train = featureData[:900]
y_train = targetData[:900]
x_test = featureData[1000:]
y_test = targetData[1000:]
print(len(x_train), len(y_train))
print(x_train[:5])

900 900
[[19  0  1  0  0  0  0  1]
 [18  1  0  1  0  0  1  0]
 [28  3  0  1  0  0  1  0]
 [33  0  0  1  0  1  0  0]
 [32  0  0  1  0  1  0  0]]


In [ ]:
# Default parameters
learning_rate = 0.0003
epochs = 5

# model -1 : Python

In [ ]:
class LinearRegressionModel:
  def __init__(self, learning_rate=0.3, epochs=10):
    self.learning_rate = learning_rate
    self.epochs = epochs
    self.lossValue = None
    self.optimalweight = None
    self.optimalbias = None
  
  def fit(self, x=[],y=[]):
    self.Sn = len(x) # number of samples
    self.Xn = len(x[0]) # number of features
    # init weight and bias
    self.wgt = np.zeros(self.Xn)
    self.bias = 0
    self.X = x
    self.Y = y
    print("Initial values: ",self.Sn,self.Xn,self.wgt, self.bias)
    for i in range(self.epochs):
      y_pred = self.predict(x)
      self.wgt = self.updated_weights(x,y,y_pred)
      self.bias = self.bias- (self.learning_rate* self.partialDerivativebias())  

  def updated_weights(self,x,y,y_pred):
    # weight = weight - L* dw
    newWeight = self.wgt.copy()
    for i in range(len(self.wgt)):
      newWeight[i] = newWeight[i]- (self.learning_rate* self.partialDerivativeWeight(i,x,y,y_pred))
    print("{}  ==>  {}".format(self.wgt, newWeight))
    return newWeight


  def partialDerivativeWeight(self,weightIndex=0, X=[], actual=[], predict =[]):
    result = 0
    sum =0
    n = len(actual)
    if (len(actual) != len(predict)) or n==0:
      return 0
    for i in range(len(actual)):
      sum = sum + (X[i][weightIndex]*(actual[i]-predict[i]))
    result = (-2/n)* sum
    return result

  def partialDerivativebias(self, actual=[], predict=[]):
    result = 0
    sum = 0
    n = len(actual)
    if (n!=len(predict)) or n==0:
      return 0
    for i in range(n):
      sum = sum + (actual[i]-predict[i])
    result = (-2/n)*sum
    return result

  def predict(self,X):
    n = len(X)
    y_pred = []
    _weight= self.wgt 
    _bias = self.bias
    for i in range(n):
      pred = _bias
      for j in range(self.Xn):
        pred = pred+ X[i][j] *_weight[j]
      y_pred.append(pred)
    return y_pred

  
  def meanSquaredError(self,actual=[],predict=[]):
    totalLoss = 0
    error = 0
    n = len(actual)
    if n == len(predict):
      for i in range(n):
        totalLoss = totalLoss + ((actual[i]-predict[i])**2)
      error = totalLoss/n
    return error

  def getParams(self):
    return {
        "weights":self.wgt,
        "bias":self.bias
    }

In [ ]:
# train 
model = LinearRegressionModel(learning_rate, epochs)
model.fit(x_train,y_train)
y_pred = model.predict(x_test)
model1_err = mean_squared_error(y_test,y_pred)
print(model.getParams())

Initial values:  900 8 [0. 0. 0. 0. 0. 0. 0. 0.] 0
[0. 0. 0. 0. 0. 0. 0. 0.]  ==>  [346.76051983   8.91852036   3.86755535   4.03510003   2.00847101
   1.70861516   2.42082332   1.76474589]
[346.76051983   8.91852036   3.86755535   4.03510003   2.00847101
   1.70861516   2.42082332   1.76474589]  ==>  [324.15812032   8.89195582   3.55001867   4.00191234   1.95141811
   1.48116016   2.584157     1.53519573]
[324.15812032   8.89195582   3.55001867   4.00191234   1.95141811
   1.48116016   2.584157     1.53519573]  ==>  [325.61397106   9.4475137    3.50510061   4.23363854   2.02888844
   1.37982572   2.89447452   1.43555047]
[325.61397106   9.4475137    3.50510061   4.23363854   2.02888844
   1.37982572   2.89447452   1.43555047]  ==>  [325.50281494   9.96464541   3.4424508    4.44803815   2.09758322
   1.27029226   3.19516193   1.32745154]
[325.50281494   9.96464541   3.4424508    4.44803815   2.09758322
   1.27029226   3.19516193   1.32745154]  ==>  [325.49373803  10.48377048   3.380981